# Hit prediction

In [1]:
# Import libraries
import pandas as pd
import os
import re
import billboard
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
from langdetect import detect
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score
pd.options.mode.chained_assignment = None

In [2]:
# Spotify authorization
spotify_client_id = os.environ.get('spotify_client_id')
spotify_client_secret = os.environ.get('spotify_client_secret')
client_credentials_manager = SpotifyClientCredentials(client_id = spotify_client_id, client_secret = spotify_client_secret)
sp = spotipy.Spotify(client_credentials_manager = client_credentials_manager)

### Last week Billboard data

In [3]:
# Download data from last week
newest_chart = billboard.ChartData('hot-100')

In [4]:
# Creating Data Frame for data from last week
song_titles, artists, positions = [[] for i in range(3)]

position = 0
for i in newest_chart:
    position += 1
    positions.append(position)
    song = str(i).split(' by ')
    song_titles.append(song[0].strip("\'"))
    artists.append(song[1])

In [5]:
billboard_weekly_wrap_up = pd.DataFrame({'Position': positions, 'Artist': artists, 'Song Title': song_titles})
display(billboard_weekly_wrap_up)

,Position,Artist,Song Title
0,1,Drake Featuring 21 Savage,Jimmy Cooks
1,2,Harry Styles,As It Was
2,3,Jack Harlow,First Class
3,4,Future Featuring Drake & Tems,Wait For U
4,5,Lizzo,About Damn Time
...,...,...,...
95,96,Kendrick Lamar,N95
96,97,Sam Smith,Love Me More
97,98,Dylan Scott,New Truck
98,99,Tate McRae,She's All I Wanna Be


In [6]:
# Spliting authors by 'and', 'featuring', 'feat.', Featuring', 'Feat.', '&' to make api searching easier and more efficient
artists = billboard_weekly_wrap_up['Artist'].to_list()
split_artists = []
for artist in artists:
    if any(re.findall(r'and|featuring|feat.|Featuring|Feat.|&|X', str(artist))):
        result = re.split(r'and|featuring|feat.|Featuring|Feat.|&|X', str(artist))[0].strip()
        split_artists.append(result)
    else:
        split_artists.append(artist)

billboard_weekly_wrap_up['Split Names'] = split_artists
display(billboard_weekly_wrap_up)

,Position,Artist,Song Title,Split Names
0,1,Drake Featuring 21 Savage,Jimmy Cooks,Drake
1,2,Harry Styles,As It Was,Harry Styles
2,3,Jack Harlow,First Class,Jack Harlow
3,4,Future Featuring Drake & Tems,Wait For U,Future
4,5,Lizzo,About Damn Time,Lizzo
...,...,...,...,...
95,96,Kendrick Lamar,N95,Kendrick Lamar
96,97,Sam Smith,Love Me More,Sam Smith
97,98,Dylan Scott,New Truck,Dylan Scott
98,99,Tate McRae,She's All I Wanna Be,Tate McRae


### Spotify data for Billboard songs

In [7]:
# Get Spotify ID
def get_spotify_ID(artist, track):
    sp_id_list = []
    music_genre_list = []
    audio_features = {}
    try:
        track_id = sp.search(q = 'artist:' + artist + ' track:' + track, type = 'track')
        sp_id = track_id['tracks']['items'][0]['id']
    except IndexError:
        sp_id = ''
    return sp_id
        
spotify_ids = []
for i in billboard_weekly_wrap_up.index:
    data = get_spotify_ID(billboard_weekly_wrap_up['Split Names'][i], billboard_weekly_wrap_up['Song Title'][i])
    spotify_ids.append(data)
    
billboard_weekly_wrap_up['Spotify ID'] = spotify_ids

In [8]:
sum(billboard_weekly_wrap_up['Spotify ID'] == '')

0

In [9]:
billboard_weekly_wrap_up = billboard_weekly_wrap_up[billboard_weekly_wrap_up['Spotify ID'] != '']
display(billboard_weekly_wrap_up)

,Position,Artist,Song Title,Split Names,Spotify ID
0,1,Drake Featuring 21 Savage,Jimmy Cooks,Drake,3F5CgOj3wFlRv51JsHbxhe
1,2,Harry Styles,As It Was,Harry Styles,4LRPiXqCikLlN15c3yImP7
2,3,Jack Harlow,First Class,Jack Harlow,1rDQ4oMwGJI7B4tovsBOxc
3,4,Future Featuring Drake & Tems,Wait For U,Future,59nOXPmaKlBfGMDeOVGrIK
4,5,Lizzo,About Damn Time,Lizzo,1PckUlxKqWQs3RlWXVBLw3
...,...,...,...,...,...
95,96,Kendrick Lamar,N95,Kendrick Lamar,0fX4oNGBWO3dSGUZcVdVV2
96,97,Sam Smith,Love Me More,Sam Smith,16MQETUgqXwY4lZcyRafU3
97,98,Dylan Scott,New Truck,Dylan Scott,4MDYIbU6I29Gjiu7JATTIJ
98,99,Tate McRae,She's All I Wanna Be,Tate McRae,0IuVhCflrQPMGRrOyoY5RW


In [10]:
# Function returns audio features for specific Spotify ID
def get_audio_features(spotify_id):
    af = pd.DataFrame(sp.audio_features(spotify_id))
    return af

temp_audio_features_list = []
for i in billboard_weekly_wrap_up['Spotify ID']:
    temp_song_audio_features = get_audio_features(i)
    temp_audio_features_list.append(temp_song_audio_features)
temp_audio_features_df = pd.concat(temp_audio_features_list)
billboard_weekly_wrap_up = billboard_weekly_wrap_up.merge(temp_audio_features_df, how = 'inner', left_on = 'Spotify ID', right_on = 'id')
display(billboard_weekly_wrap_up)

,Position,Artist,Song Title,Split Names,Spotify ID,danceability,energy,key,loudness,mode,...,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature
0,1,Drake Featuring 21 Savage,Jimmy Cooks,Drake,3F5CgOj3wFlRv51JsHbxhe,0.529,0.673,0,-4.711,1,...,0.0930,0.366,165.921,audio_features,3F5CgOj3wFlRv51JsHbxhe,spotify:track:3F5CgOj3wFlRv51JsHbxhe,https://api.spotify.com/v1/tracks/3F5CgOj3wFlR...,https://api.spotify.com/v1/audio-analysis/3F5C...,218365,4
1,2,Harry Styles,As It Was,Harry Styles,4LRPiXqCikLlN15c3yImP7,0.520,0.731,6,-5.338,0,...,0.3110,0.662,173.930,audio_features,4LRPiXqCikLlN15c3yImP7,spotify:track:4LRPiXqCikLlN15c3yImP7,https://api.spotify.com/v1/tracks/4LRPiXqCikLl...,https://api.spotify.com/v1/audio-analysis/4LRP...,167303,4
2,3,Jack Harlow,First Class,Jack Harlow,1rDQ4oMwGJI7B4tovsBOxc,0.905,0.563,8,-6.135,1,...,0.1130,0.324,106.998,audio_features,1rDQ4oMwGJI7B4tovsBOxc,spotify:track:1rDQ4oMwGJI7B4tovsBOxc,https://api.spotify.com/v1/tracks/1rDQ4oMwGJI7...,https://api.spotify.com/v1/audio-analysis/1rDQ...,173948,4
3,4,Future Featuring Drake & Tems,Wait For U,Future,59nOXPmaKlBfGMDeOVGrIK,0.463,0.642,1,-4.474,1,...,0.0686,0.339,83.389,audio_features,59nOXPmaKlBfGMDeOVGrIK,spotify:track:59nOXPmaKlBfGMDeOVGrIK,https://api.spotify.com/v1/tracks/59nOXPmaKlBf...,https://api.spotify.com/v1/audio-analysis/59nO...,189893,4
4,5,Lizzo,About Damn Time,Lizzo,1PckUlxKqWQs3RlWXVBLw3,0.836,0.743,10,-6.305,0,...,0.3350,0.722,108.966,audio_features,1PckUlxKqWQs3RlWXVBLw3,spotify:track:1PckUlxKqWQs3RlWXVBLw3,https://api.spotify.com/v1/tracks/1PckUlxKqWQs...,https://api.spotify.com/v1/audio-analysis/1Pck...,191822,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,96,Kendrick Lamar,N95,Kendrick Lamar,0fX4oNGBWO3dSGUZcVdVV2,0.790,0.670,1,-5.527,1,...,0.1190,0.408,139.956,audio_features,0fX4oNGBWO3dSGUZcVdVV2,spotify:track:0fX4oNGBWO3dSGUZcVdVV2,https://api.spotify.com/v1/tracks/0fX4oNGBWO3d...,https://api.spotify.com/v1/audio-analysis/0fX4...,195950,4
96,97,Sam Smith,Love Me More,Sam Smith,16MQETUgqXwY4lZcyRafU3,0.791,0.439,10,-5.620,1,...,0.1260,0.559,85.004,audio_features,16MQETUgqXwY4lZcyRafU3,spotify:track:16MQETUgqXwY4lZcyRafU3,https://api.spotify.com/v1/tracks/16MQETUgqXwY...,https://api.spotify.com/v1/audio-analysis/16MQ...,190659,4
97,98,Dylan Scott,New Truck,Dylan Scott,4MDYIbU6I29Gjiu7JATTIJ,0.529,0.755,1,-5.864,1,...,0.0986,0.528,96.860,audio_features,4MDYIbU6I29Gjiu7JATTIJ,spotify:track:4MDYIbU6I29Gjiu7JATTIJ,https://api.spotify.com/v1/tracks/4MDYIbU6I29G...,https://api.spotify.com/v1/audio-analysis/4MDY...,174360,4
98,99,Tate McRae,She's All I Wanna Be,Tate McRae,0IuVhCflrQPMGRrOyoY5RW,0.614,0.644,2,-5.372,0,...,0.1170,0.651,160.036,audio_features,0IuVhCflrQPMGRrOyoY5RW,spotify:track:0IuVhCflrQPMGRrOyoY5RW,https://api.spotify.com/v1/tracks/0IuVhCflrQPM...,https://api.spotify.com/v1/audio-analysis/0IuV...,206772,4


In [11]:
# Drop duplicates
billboard_weekly_wrap_up = billboard_weekly_wrap_up.drop_duplicates()
display(billboard_weekly_wrap_up)

,Position,Artist,Song Title,Split Names,Spotify ID,danceability,energy,key,loudness,mode,...,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature
0,1,Drake Featuring 21 Savage,Jimmy Cooks,Drake,3F5CgOj3wFlRv51JsHbxhe,0.529,0.673,0,-4.711,1,...,0.0930,0.366,165.921,audio_features,3F5CgOj3wFlRv51JsHbxhe,spotify:track:3F5CgOj3wFlRv51JsHbxhe,https://api.spotify.com/v1/tracks/3F5CgOj3wFlR...,https://api.spotify.com/v1/audio-analysis/3F5C...,218365,4
1,2,Harry Styles,As It Was,Harry Styles,4LRPiXqCikLlN15c3yImP7,0.520,0.731,6,-5.338,0,...,0.3110,0.662,173.930,audio_features,4LRPiXqCikLlN15c3yImP7,spotify:track:4LRPiXqCikLlN15c3yImP7,https://api.spotify.com/v1/tracks/4LRPiXqCikLl...,https://api.spotify.com/v1/audio-analysis/4LRP...,167303,4
2,3,Jack Harlow,First Class,Jack Harlow,1rDQ4oMwGJI7B4tovsBOxc,0.905,0.563,8,-6.135,1,...,0.1130,0.324,106.998,audio_features,1rDQ4oMwGJI7B4tovsBOxc,spotify:track:1rDQ4oMwGJI7B4tovsBOxc,https://api.spotify.com/v1/tracks/1rDQ4oMwGJI7...,https://api.spotify.com/v1/audio-analysis/1rDQ...,173948,4
3,4,Future Featuring Drake & Tems,Wait For U,Future,59nOXPmaKlBfGMDeOVGrIK,0.463,0.642,1,-4.474,1,...,0.0686,0.339,83.389,audio_features,59nOXPmaKlBfGMDeOVGrIK,spotify:track:59nOXPmaKlBfGMDeOVGrIK,https://api.spotify.com/v1/tracks/59nOXPmaKlBf...,https://api.spotify.com/v1/audio-analysis/59nO...,189893,4
4,5,Lizzo,About Damn Time,Lizzo,1PckUlxKqWQs3RlWXVBLw3,0.836,0.743,10,-6.305,0,...,0.3350,0.722,108.966,audio_features,1PckUlxKqWQs3RlWXVBLw3,spotify:track:1PckUlxKqWQs3RlWXVBLw3,https://api.spotify.com/v1/tracks/1PckUlxKqWQs...,https://api.spotify.com/v1/audio-analysis/1Pck...,191822,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,96,Kendrick Lamar,N95,Kendrick Lamar,0fX4oNGBWO3dSGUZcVdVV2,0.790,0.670,1,-5.527,1,...,0.1190,0.408,139.956,audio_features,0fX4oNGBWO3dSGUZcVdVV2,spotify:track:0fX4oNGBWO3dSGUZcVdVV2,https://api.spotify.com/v1/tracks/0fX4oNGBWO3d...,https://api.spotify.com/v1/audio-analysis/0fX4...,195950,4
96,97,Sam Smith,Love Me More,Sam Smith,16MQETUgqXwY4lZcyRafU3,0.791,0.439,10,-5.620,1,...,0.1260,0.559,85.004,audio_features,16MQETUgqXwY4lZcyRafU3,spotify:track:16MQETUgqXwY4lZcyRafU3,https://api.spotify.com/v1/tracks/16MQETUgqXwY...,https://api.spotify.com/v1/audio-analysis/16MQ...,190659,4
97,98,Dylan Scott,New Truck,Dylan Scott,4MDYIbU6I29Gjiu7JATTIJ,0.529,0.755,1,-5.864,1,...,0.0986,0.528,96.860,audio_features,4MDYIbU6I29Gjiu7JATTIJ,spotify:track:4MDYIbU6I29Gjiu7JATTIJ,https://api.spotify.com/v1/tracks/4MDYIbU6I29G...,https://api.spotify.com/v1/audio-analysis/4MDY...,174360,4
98,99,Tate McRae,She's All I Wanna Be,Tate McRae,0IuVhCflrQPMGRrOyoY5RW,0.614,0.644,2,-5.372,0,...,0.1170,0.651,160.036,audio_features,0IuVhCflrQPMGRrOyoY5RW,spotify:track:0IuVhCflrQPMGRrOyoY5RW,https://api.spotify.com/v1/tracks/0IuVhCflrQPM...,https://api.spotify.com/v1/audio-analysis/0IuV...,206772,4


### Load data

In [12]:
# Load gathered data
all_songs = pd.read_csv('spotify_data.csv')
# Limit data
songs = all_songs[all_songs['Year'] > 2004]

In [13]:
songs['Is hit'] = [1 if i <= 10 else 0 for i in songs['Position']]

In [14]:
# Count hits and non-hits
songs['Is hit'].value_counts()

0    1131
1     138
Name: Is hit, dtype: int64

### Building model

In [15]:
# Split data into train and test subsets
X = songs[['acousticness', 'danceability', 'duration_ms', 'energy', 'instrumentalness', 'key', 'liveness', 'loudness', 'mode', 'speechiness', 'tempo', 'time_signature', 'valence']]
y = songs['Is hit']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25, random_state = 55)

print('Train data shape: {}'.format(X_train.shape))
print('Test data shape: {}'.format(X_test.shape))

Train data shape: (951, 13)
Test data shape: (318, 13)


In [16]:
# Handling imbalanced train data with smote
oversample = SMOTE()
X_smote, y_smote = oversample.fit_resample(X_train, y_train)

In [17]:
print(X_smote.shape, y_smote.shape)

(1684, 13) (1684,)


In [18]:
# Random forest model
random_forest = RandomForestClassifier(n_estimators = 2000, random_state = 3)
random_forest.fit(X_smote, y_smote)
y_pred = random_forest.predict(X_test)

In [19]:
# Count predicted values
pd.Series(y_pred).value_counts()

0    290
1     28
dtype: int64

### Evaluation - test data

In [20]:
print("Accuracy score: ", accuracy_score(y_test, y_pred))

Accuracy score:  0.8333333333333334


In [21]:
print("Recall score: ", recall_score(y_test, y_pred))

Recall score:  0.06896551724137931


In [22]:
print("Precision score: ", precision_score(y_test, y_pred))

Precision score:  0.07142857142857142


In [23]:
print("F1 score: ", f1_score(y_test, y_pred))

F1 score:  0.07017543859649124


### Evaluation - train data

In [24]:
# Train data
y_pred_train = random_forest.predict(X_smote)

In [25]:
print("Accuracy score train: ", accuracy_score(y_smote, y_pred_train))

Accuracy score train:  0.9910926365795725


In [26]:
print("Recall score train: ", recall_score(y_smote, y_pred_train))

Recall score train:  0.9916864608076009


In [27]:
print("Precision score train: ", precision_score(y_smote, y_pred_train))

Precision score train:  0.9905100830367735


In [28]:
print("F1 score train: ", f1_score(y_smote, y_pred_train))

F1 score train:  0.9910979228486646


In [29]:
# Data from the last week
display(billboard_weekly_wrap_up)

,Position,Artist,Song Title,Split Names,Spotify ID,danceability,energy,key,loudness,mode,...,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature
0,1,Drake Featuring 21 Savage,Jimmy Cooks,Drake,3F5CgOj3wFlRv51JsHbxhe,0.529,0.673,0,-4.711,1,...,0.0930,0.366,165.921,audio_features,3F5CgOj3wFlRv51JsHbxhe,spotify:track:3F5CgOj3wFlRv51JsHbxhe,https://api.spotify.com/v1/tracks/3F5CgOj3wFlR...,https://api.spotify.com/v1/audio-analysis/3F5C...,218365,4
1,2,Harry Styles,As It Was,Harry Styles,4LRPiXqCikLlN15c3yImP7,0.520,0.731,6,-5.338,0,...,0.3110,0.662,173.930,audio_features,4LRPiXqCikLlN15c3yImP7,spotify:track:4LRPiXqCikLlN15c3yImP7,https://api.spotify.com/v1/tracks/4LRPiXqCikLl...,https://api.spotify.com/v1/audio-analysis/4LRP...,167303,4
2,3,Jack Harlow,First Class,Jack Harlow,1rDQ4oMwGJI7B4tovsBOxc,0.905,0.563,8,-6.135,1,...,0.1130,0.324,106.998,audio_features,1rDQ4oMwGJI7B4tovsBOxc,spotify:track:1rDQ4oMwGJI7B4tovsBOxc,https://api.spotify.com/v1/tracks/1rDQ4oMwGJI7...,https://api.spotify.com/v1/audio-analysis/1rDQ...,173948,4
3,4,Future Featuring Drake & Tems,Wait For U,Future,59nOXPmaKlBfGMDeOVGrIK,0.463,0.642,1,-4.474,1,...,0.0686,0.339,83.389,audio_features,59nOXPmaKlBfGMDeOVGrIK,spotify:track:59nOXPmaKlBfGMDeOVGrIK,https://api.spotify.com/v1/tracks/59nOXPmaKlBf...,https://api.spotify.com/v1/audio-analysis/59nO...,189893,4
4,5,Lizzo,About Damn Time,Lizzo,1PckUlxKqWQs3RlWXVBLw3,0.836,0.743,10,-6.305,0,...,0.3350,0.722,108.966,audio_features,1PckUlxKqWQs3RlWXVBLw3,spotify:track:1PckUlxKqWQs3RlWXVBLw3,https://api.spotify.com/v1/tracks/1PckUlxKqWQs...,https://api.spotify.com/v1/audio-analysis/1Pck...,191822,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,96,Kendrick Lamar,N95,Kendrick Lamar,0fX4oNGBWO3dSGUZcVdVV2,0.790,0.670,1,-5.527,1,...,0.1190,0.408,139.956,audio_features,0fX4oNGBWO3dSGUZcVdVV2,spotify:track:0fX4oNGBWO3dSGUZcVdVV2,https://api.spotify.com/v1/tracks/0fX4oNGBWO3d...,https://api.spotify.com/v1/audio-analysis/0fX4...,195950,4
96,97,Sam Smith,Love Me More,Sam Smith,16MQETUgqXwY4lZcyRafU3,0.791,0.439,10,-5.620,1,...,0.1260,0.559,85.004,audio_features,16MQETUgqXwY4lZcyRafU3,spotify:track:16MQETUgqXwY4lZcyRafU3,https://api.spotify.com/v1/tracks/16MQETUgqXwY...,https://api.spotify.com/v1/audio-analysis/16MQ...,190659,4
97,98,Dylan Scott,New Truck,Dylan Scott,4MDYIbU6I29Gjiu7JATTIJ,0.529,0.755,1,-5.864,1,...,0.0986,0.528,96.860,audio_features,4MDYIbU6I29Gjiu7JATTIJ,spotify:track:4MDYIbU6I29Gjiu7JATTIJ,https://api.spotify.com/v1/tracks/4MDYIbU6I29G...,https://api.spotify.com/v1/audio-analysis/4MDY...,174360,4
98,99,Tate McRae,She's All I Wanna Be,Tate McRae,0IuVhCflrQPMGRrOyoY5RW,0.614,0.644,2,-5.372,0,...,0.1170,0.651,160.036,audio_features,0IuVhCflrQPMGRrOyoY5RW,spotify:track:0IuVhCflrQPMGRrOyoY5RW,https://api.spotify.com/v1/tracks/0IuVhCflrQPM...,https://api.spotify.com/v1/audio-analysis/0IuV...,206772,4


In [30]:
# Delete songs from last week which appeared in year-end chart in 2021
cond = billboard_weekly_wrap_up['Spotify ID'].isin(songs['Spotify ID'])
billboard_weekly_wrap_up.drop(billboard_weekly_wrap_up[cond].index, inplace = True)
display(billboard_weekly_wrap_up)

,Position,Artist,Song Title,Split Names,Spotify ID,danceability,energy,key,loudness,mode,...,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature
0,1,Drake Featuring 21 Savage,Jimmy Cooks,Drake,3F5CgOj3wFlRv51JsHbxhe,0.529,0.673,0,-4.711,1,...,0.0930,0.366,165.921,audio_features,3F5CgOj3wFlRv51JsHbxhe,spotify:track:3F5CgOj3wFlRv51JsHbxhe,https://api.spotify.com/v1/tracks/3F5CgOj3wFlR...,https://api.spotify.com/v1/audio-analysis/3F5C...,218365,4
1,2,Harry Styles,As It Was,Harry Styles,4LRPiXqCikLlN15c3yImP7,0.520,0.731,6,-5.338,0,...,0.3110,0.662,173.930,audio_features,4LRPiXqCikLlN15c3yImP7,spotify:track:4LRPiXqCikLlN15c3yImP7,https://api.spotify.com/v1/tracks/4LRPiXqCikLl...,https://api.spotify.com/v1/audio-analysis/4LRP...,167303,4
2,3,Jack Harlow,First Class,Jack Harlow,1rDQ4oMwGJI7B4tovsBOxc,0.905,0.563,8,-6.135,1,...,0.1130,0.324,106.998,audio_features,1rDQ4oMwGJI7B4tovsBOxc,spotify:track:1rDQ4oMwGJI7B4tovsBOxc,https://api.spotify.com/v1/tracks/1rDQ4oMwGJI7...,https://api.spotify.com/v1/audio-analysis/1rDQ...,173948,4
3,4,Future Featuring Drake & Tems,Wait For U,Future,59nOXPmaKlBfGMDeOVGrIK,0.463,0.642,1,-4.474,1,...,0.0686,0.339,83.389,audio_features,59nOXPmaKlBfGMDeOVGrIK,spotify:track:59nOXPmaKlBfGMDeOVGrIK,https://api.spotify.com/v1/tracks/59nOXPmaKlBf...,https://api.spotify.com/v1/audio-analysis/59nO...,189893,4
4,5,Lizzo,About Damn Time,Lizzo,1PckUlxKqWQs3RlWXVBLw3,0.836,0.743,10,-6.305,0,...,0.3350,0.722,108.966,audio_features,1PckUlxKqWQs3RlWXVBLw3,spotify:track:1PckUlxKqWQs3RlWXVBLw3,https://api.spotify.com/v1/tracks/1PckUlxKqWQs...,https://api.spotify.com/v1/audio-analysis/1Pck...,191822,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,96,Kendrick Lamar,N95,Kendrick Lamar,0fX4oNGBWO3dSGUZcVdVV2,0.790,0.670,1,-5.527,1,...,0.1190,0.408,139.956,audio_features,0fX4oNGBWO3dSGUZcVdVV2,spotify:track:0fX4oNGBWO3dSGUZcVdVV2,https://api.spotify.com/v1/tracks/0fX4oNGBWO3d...,https://api.spotify.com/v1/audio-analysis/0fX4...,195950,4
96,97,Sam Smith,Love Me More,Sam Smith,16MQETUgqXwY4lZcyRafU3,0.791,0.439,10,-5.620,1,...,0.1260,0.559,85.004,audio_features,16MQETUgqXwY4lZcyRafU3,spotify:track:16MQETUgqXwY4lZcyRafU3,https://api.spotify.com/v1/tracks/16MQETUgqXwY...,https://api.spotify.com/v1/audio-analysis/16MQ...,190659,4
97,98,Dylan Scott,New Truck,Dylan Scott,4MDYIbU6I29Gjiu7JATTIJ,0.529,0.755,1,-5.864,1,...,0.0986,0.528,96.860,audio_features,4MDYIbU6I29Gjiu7JATTIJ,spotify:track:4MDYIbU6I29Gjiu7JATTIJ,https://api.spotify.com/v1/tracks/4MDYIbU6I29G...,https://api.spotify.com/v1/audio-analysis/4MDY...,174360,4
98,99,Tate McRae,She's All I Wanna Be,Tate McRae,0IuVhCflrQPMGRrOyoY5RW,0.614,0.644,2,-5.372,0,...,0.1170,0.651,160.036,audio_features,0IuVhCflrQPMGRrOyoY5RW,spotify:track:0IuVhCflrQPMGRrOyoY5RW,https://api.spotify.com/v1/tracks/0IuVhCflrQPM...,https://api.spotify.com/v1/audio-analysis/0IuV...,206772,4


### Applying model on test data

In [31]:
# Results
chart_audio_features = billboard_weekly_wrap_up[['acousticness', 'danceability', 'duration_ms', 'energy', 'instrumentalness', 'key', 'liveness', 'loudness', 'mode', 'speechiness', 'tempo', 'time_signature', 'valence']]
billboard_weekly_wrap_up['Predict as hit'] = random_forest.predict(chart_audio_features)
display(billboard_weekly_wrap_up[billboard_weekly_wrap_up['Predict as hit'] == 1])

,Position,Artist,Song Title,Split Names,Spotify ID,danceability,energy,key,loudness,mode,...,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature,Predict as hit
4,5,Lizzo,About Damn Time,Lizzo,1PckUlxKqWQs3RlWXVBLw3,0.836,0.743,10,-6.305,0,...,0.722,108.966,audio_features,1PckUlxKqWQs3RlWXVBLw3,spotify:track:1PckUlxKqWQs3RlWXVBLw3,https://api.spotify.com/v1/tracks/1PckUlxKqWQs...,https://api.spotify.com/v1/audio-analysis/1Pck...,191822,4,1
6,7,Drake,Falling Back,Drake,1vbn9fEyw1IYhqgZJdu9ZB,0.718,0.758,10,-8.290,1,...,0.349,118.989,audio_features,1vbn9fEyw1IYhqgZJdu9ZB,spotify:track:1vbn9fEyw1IYhqgZJdu9ZB,https://api.spotify.com/v1/tracks/1vbn9fEyw1IY...,https://api.spotify.com/v1/audio-analysis/1vbn...,266179,4,1
10,11,Bad Bunny & Chencho Corleone,Me Porto Bonito,Bad Bunny,6Sq7ltF9Qa7SNFBsV5Cogx,0.911,0.712,1,-5.105,0,...,0.425,92.005,audio_features,6Sq7ltF9Qa7SNFBsV5Cogx,spotify:track:6Sq7ltF9Qa7SNFBsV5Cogx,https://api.spotify.com/v1/tracks/6Sq7ltF9Qa7S...,https://api.spotify.com/v1/audio-analysis/6Sq7...,178567,4,1
15,16,Bad Bunny,Titi Me Pregunto,Bad Bunny,1IHWl5LamUGEuP4ozKQSXZ,0.650,0.715,5,-5.198,0,...,0.187,106.672,audio_features,1IHWl5LamUGEuP4ozKQSXZ,spotify:track:1IHWl5LamUGEuP4ozKQSXZ,https://api.spotify.com/v1/tracks/1IHWl5LamUGE...,https://api.spotify.com/v1/audio-analysis/1IHW...,243717,4,1
20,21,Drake,A Keeper,Drake,0nAZGkBGKQCXyaoSJfRhC1,0.600,0.482,1,-11.596,1,...,0.323,121.909,audio_features,0nAZGkBGKQCXyaoSJfRhC1,spotify:track:0nAZGkBGKQCXyaoSJfRhC1,https://api.spotify.com/v1/tracks/0nAZGkBGKQCX...,https://api.spotify.com/v1/audio-analysis/0nAZ...,173097,4,1
34,35,Elton John & Dua Lipa,Cold Heart (PNAU Remix),Elton John,6zSpb8dQRaw0M1dK8PBwQz,0.796,0.798,1,-6.312,1,...,0.942,116.032,audio_features,6zSpb8dQRaw0M1dK8PBwQz,spotify:track:6zSpb8dQRaw0M1dK8PBwQz,https://api.spotify.com/v1/tracks/6zSpb8dQRaw0...,https://api.spotify.com/v1/audio-analysis/6zSp...,202735,4,1
39,40,Karol G,Provenza,Karol G,7dSZ6zGTQx66c2GF91xCrb,0.870,0.516,1,-8.006,1,...,0.530,111.005,audio_features,7dSZ6zGTQx66c2GF91xCrb,spotify:track:7dSZ6zGTQx66c2GF91xCrb,https://api.spotify.com/v1/tracks/7dSZ6zGTQx66...,https://api.spotify.com/v1/audio-analysis/7dSZ...,210200,4,1
61,62,Drake,Down Hill,Drake,3N3d4OWbPVxTQVBFqNyykk,0.718,0.319,3,-11.261,1,...,0.639,109.994,audio_features,3N3d4OWbPVxTQVBFqNyykk,spotify:track:3N3d4OWbPVxTQVBFqNyykk,https://api.spotify.com/v1/tracks/3N3d4OWbPVxT...,https://api.spotify.com/v1/audio-analysis/3N3d...,250865,4,1
73,74,JNR CHOI & Sam Tompkins,To The Moon!,JNR CHOI,5vUnjhBzRJJIAOJPde6zDx,0.745,0.650,2,-11.814,1,...,0.386,144.047,audio_features,5vUnjhBzRJJIAOJPde6zDx,spotify:track:5vUnjhBzRJJIAOJPde6zDx,https://api.spotify.com/v1/tracks/5vUnjhBzRJJI...,https://api.spotify.com/v1/audio-analysis/5vUn...,152137,4,1
76,77,Russell Dickerson & Jake Scott,She Likes It,Russell Dickerson,6VVqPBFZIbcmv14zNfOc13,0.790,0.325,4,-11.689,1,...,0.539,126.877,audio_features,6VVqPBFZIbcmv14zNfOc13,spotify:track:6VVqPBFZIbcmv14zNfOc13,https://api.spotify.com/v1/tracks/6VVqPBFZIbcm...,https://api.spotify.com/v1/audio-analysis/6VVq...,163311,4,1
